In [1]:
from citipy import citipy
import pandas as pd
import numpy as np
import json
import requests
import matplotlib.pyplot as plt
from config import weather_api
from pprint import pprint

# Creating the cities DataFrame

In [3]:
#Create a list of random latitudes and longitudes
cities_df = pd.DataFrame({"lat":np.random.uniform(-90,90,500), "long":np.random.uniform(-180,180,500)})

print(len(cities_df))

cities_df.head()

500


,lat,long
0,22.069837,139.812016
1,-44.007901,31.664782
2,31.324890,15.907863
3,88.971471,153.363336
4,-74.338884,23.201030


In [4]:
city_name = []
country_list =[]

for coords in range(0, len(cities_df["lat"])):
    lat=cities_df.iloc[coords]["lat"]
    long=cities_df.iloc[coords]["long"]
    city = citipy.nearest_city(lat,long)
    name= city.city_name
    city_name.append(name)
    country= city.country_code
    country_list.append(country)

cities_df["city"]=city_name
cities_df["country"]=country_list

cities_df.head()

,lat,long,city,country
0,22.069837,139.812016,naze,jp
1,-44.007901,31.664782,port alfred,za
2,31.324890,15.907863,surt,ly
3,88.971471,153.363336,chokurdakh,ru
4,-74.338884,23.201030,bredasdorp,za


In [5]:
cities_df["city"].duplicated().value_counts()

False    281
True     219
Name: city, dtype: int64

### Since there are duplicated cities in our DF, we  have to drop those specific rows. 

In [14]:
cleancities_df = cities_df.drop_duplicates(subset="city", keep="first").reset_index(drop=True)


In [15]:
len(cleancities_df)

281

### Now that we have a clean data set, we can keep going with the exercise.

In [16]:
cleancities_df["max temp"] = ""
cleancities_df["humidity"] = ""
cleancities_df["cloudiness"] = ""
cleancities_df["wind speed"] = ""
cleancities_df["date"] = ""

cleancities_df.head()

,lat,long,city,country,max temp,humidity,cloudiness,wind speed,date
0,22.069837,139.812016,naze,jp,,,,,
1,-44.007901,31.664782,port alfred,za,,,,,
2,31.324890,15.907863,surt,ly,,,,,
3,88.971471,153.363336,chokurdakh,ru,,,,,
4,-74.338884,23.201030,bredasdorp,za,,,,,


In [17]:
url = "http://api.openweathermap.org/data/2.5/weather?"
units = "imperial"
api_key = weather_api
query_url = f"{url}appid={api_key}&units={units}&q="

for index, row in cleancities_df.iterrows():
    city = row["city"]
    response = requests.get(query_url + city).json()
    
    print(f"Looking for the weather in city number {index}, {city}")
          
    try:
          cleancities_df.at[index, "max temp"] = response["main"]["temp_max"]
          cleancities_df.at[index, "humidity"] = response["main"]["humidity"]
          cleancities_df.at[index, "cloudiness"] = response["clouds"]["all"]
          cleancities_df.at[index, "wind speed"] = response["wind"]["speed"]
          cleancities_df.at[index, "date"] = response["dt"]
    except:
          print("Missing city. Moving on...")
          

cleancities_df.head()

Looking for the weather in city number 0, naze
Looking for the weather in city number 1, port alfred
Looking for the weather in city number 2, surt
Looking for the weather in city number 3, chokurdakh
Looking for the weather in city number 4, bredasdorp
Looking for the weather in city number 5, avarua
Looking for the weather in city number 6, busselton
Looking for the weather in city number 7, zhoucheng
Looking for the weather in city number 8, angoche
Looking for the weather in city number 9, port hawkesbury
Looking for the weather in city number 10, yellowknife
Looking for the weather in city number 11, concepcion del oro
Looking for the weather in city number 12, olafsvik
Looking for the weather in city number 13, ancud
Looking for the weather in city number 14, kirakira
Looking for the weather in city number 15, san carlos de bariloche
Looking for the weather in city number 16, hilo
Looking for the weather in city number 17, muyezerskiy
Looking for the weather in city number 18, ri

Looking for the weather in city number 147, krasnoselkup
Looking for the weather in city number 148, kita
Looking for the weather in city number 149, naryan-mar
Looking for the weather in city number 150, sao filipe
Looking for the weather in city number 151, mareeba
Looking for the weather in city number 152, astoria
Looking for the weather in city number 153, castro
Looking for the weather in city number 154, macherla
Looking for the weather in city number 155, litoral del san juan
Missing city. Moving on...
Looking for the weather in city number 156, bastia
Looking for the weather in city number 157, tarbes
Looking for the weather in city number 158, acapulco
Looking for the weather in city number 159, jalu
Looking for the weather in city number 160, aklavik
Looking for the weather in city number 161, shu
Looking for the weather in city number 162, kieta
Looking for the weather in city number 163, tsihombe
Missing city. Moving on...
Looking for the weather in city number 164, butari

,lat,long,city,country,max temp,humidity,cloudiness,wind speed,date
0,22.069837,139.812016,naze,jp,84.2,83,75,6.93,1599090113
1,-44.007901,31.664782,port alfred,za,64.33,69,42,9.01,1599090113
2,31.324890,15.907863,surt,ly,77.99,82,0,15.7,1599090113
3,88.971471,153.363336,chokurdakh,ru,42.6,66,78,10.42,1599090114
4,-74.338884,23.201030,bredasdorp,za,60.8,72,63,12.75,1599090114
